In [1]:
pip install sqlalchemy fastapi uvicorn


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.1 MB 1.9 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.1 MB 2.4 MB/s eta 0:00:01
   ----------------------------------- ---- 1.8/2.1 MB 2.4 MB/s eta 0:00:01
   ----------------------------------- ---- 1.8/2.1 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/1.9 MB 2.0 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/1.9 MB 2.0 MB/s eta 0:00:01
   --------------------------- ------------ 1.3/1.9 MB 1.9 MB/s eta 0:00:01
   -------------------------------------- - 1.8/1.9 MB 1.9 MB/s eta 0:00:01
   ------------------------------------

In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

# Define the database
DATABASE_URL = "sqlite:///bank.db"  # Use SQLite for simplicity

# Set up SQLAlchemy
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Define the Transaction Model
class Transaction(Base):
    __tablename__ = 'transactions'
    id = Column(Integer, primary_key=True, index=True)
    account_id = Column(Integer, ForeignKey('accounts.id'))
    type = Column(String)  # "deposit" or "withdrawal"
    amount = Column(Float)

# Define the Bank Account Model
class Account(Base):
    __tablename__ = 'accounts'
    id = Column(Integer, primary_key=True, index=True)
    account_holder = Column(String, unique=True, index=True)
    balance = Column(Float, default=0.0)
    transactions = relationship("Transaction", back_populates="account")

Transaction.account = relationship("Account", back_populates="transactions")

# Create tables
Base.metadata.create_all(bind=engine)


C:\Users\abdul.samad\AppData\Local\Temp\ipykernel_5192\1508111421.py:11: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [4]:
class BankAccount:
    def __init__(self, account_holder: str):
        self.session = SessionLocal()
        
        # Check if account exists; if not, create a new one
        account = self.session.query(Account).filter_by(account_holder=account_holder).first()
        if account is None:
            account = Account(account_holder=account_holder, balance=0.0)
            self.session.add(account)
            self.session.commit()
        self.account = account

    def deposit(self, amount: float):
        self.account.balance += amount
        transaction = Transaction(account_id=self.account.id, type="deposit", amount=amount)
        self.session.add(transaction)
        self.session.commit()
        print(f"Deposited {amount}. New balance is {self.account.balance}.")

    def withdraw(self, amount: float):
        if self.account.balance >= amount:
            self.account.balance -= amount
            transaction = Transaction(account_id=self.account.id, type="withdrawal", amount=amount)
            self.session.add(transaction)
            self.session.commit()
            print(f"Withdrew {amount}. New balance is {self.account.balance}.")
        else:
            print("Insufficient funds.")

    def display_balance(self):
        print(f"Current balance: {self.account.balance}")

    def get_transaction_history(self):
        transactions = self.session.query(Transaction).filter_by(account_id=self.account.id).all()
        for t in transactions:
            print(f"Transaction {t.type} of amount {t.amount}")


In [8]:
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

# Define the database
DATABASE_URL = "sqlite:///bank.db"  # SQLite for simplicity

# Set up SQLAlchemy
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Define the Transaction Model
class Transaction(Base):
    __tablename__ = 'transactions'
    id = Column(Integer, primary_key=True, index=True)
    account_id = Column(Integer, ForeignKey('accounts.id'))
    type = Column(String)  # "deposit" or "withdrawal"
    amount = Column(Float)

# Define the Bank Account Model
class Account(Base):
    __tablename__ = 'accounts'
    id = Column(Integer, primary_key=True, index=True)
    account_holder = Column(String, unique=True, index=True)
    balance = Column(Float, default=0.0)
    transactions = relationship("Transaction", back_populates="account")

Transaction.account = relationship("Account", back_populates="transactions")

# Create tables
Base.metadata.create_all(bind=engine)

# Define the BankAccount class
class BankAccount:
    def __init__(self, account_holder: str):
        self.session = SessionLocal()
        
        # Check if account exists; if not, create a new one
        account = self.session.query(Account).filter_by(account_holder=account_holder).first()
        if account is None:
            account = Account(account_holder=account_holder, balance=0.0)
            self.session.add(account)
            self.session.commit()
        self.account = account

    def deposit(self, amount: float):
        self.account.balance += amount
        transaction = Transaction(account_id=self.account.id, type="deposit", amount=amount)
        self.session.add(transaction)
        self.session.commit()
        print(f"Deposited {amount}. New balance is {self.account.balance}.")

    def withdraw(self, amount: float):
        if self.account.balance >= amount:
            self.account.balance -= amount
            transaction = Transaction(account_id=self.account.id, type="withdrawal", amount=amount)
            self.session.add(transaction)
            self.session.commit()
            print(f"Withdrew {amount}. New balance is {self.account.balance}.")
        else:
            print("Insufficient funds.")

    def display_balance(self):
        print(f"Current balance: {self.account.balance}")

    def get_transaction_history(self):
        transactions = self.session.query(Transaction).filter_by(account_id=self.account.id).all()
        for t in transactions:
            print(f"Transaction {t.type} of amount {t.amount}")

# Test the BankAccount Class

# Step 1: Create an account for "John Doe"
account = BankAccount("John Doe")

# Step 2: Deposit funds
account.deposit(100.0)  # Depositing $100
account.deposit(50.0)   # Depositing another $50

# Step 3: Withdraw funds
account.withdraw(30.0)  # Withdrawing $30
account.withdraw(150.0) # Trying to withdraw $150 (should fail if balance is insufficient)

# Step 4: Display the balance
account.display_balance()

# Step 5: Get transaction history
account.get_transaction_history()


C:\Users\abdul.samad\AppData\Local\Temp\ipykernel_5192\908804092.py:11: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Deposited 100.0. New balance is 100.0.
Deposited 50.0. New balance is 150.0.
Withdrew 30.0. New balance is 120.0.
Insufficient funds.
Current balance: 120.0
Transaction deposit of amount 100.0
Transaction deposit of amount 50.0
Transaction withdrawal of amount 30.0


In [9]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI()

class DepositRequest(BaseModel):
    amount: float

class WithdrawRequest(BaseModel):
    amount: float

@app.post("/account/create/{account_holder}")
def create_account(account_holder: str):
    session = SessionLocal()
    existing_account = session.query(Account).filter_by(account_holder=account_holder).first()
    if existing_account:
        raise HTTPException(status_code=400, detail="Account already exists")
    account = Account(account_holder=account_holder, balance=0.0)
    session.add(account)
    session.commit()
    return {"message": "Account created successfully", "account_holder": account_holder}

@app.post("/account/deposit/{account_holder}")
def deposit(account_holder: str, request: DepositRequest):
    try:
        bank_account = BankAccount(account_holder)
        bank_account.deposit(request.amount)
        return {"message": f"Deposited {request.amount}", "new_balance": bank_account.get_balance()}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.post("/account/withdraw/{account_holder}")
def withdraw(account_holder: str, request: WithdrawRequest):
    try:
        bank_account = BankAccount(account_holder)
        bank_account.withdraw(request.amount)
        return {"message": f"Withdrew {request.amount}", "new_balance": bank_account.get_balance()}
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.get("/account/balance/{account_holder}")
def get_balance(account_holder: str):
    bank_account = BankAccount(account_holder)
    return {"account_holder": account_holder, "balance": bank_account.get_balance()}

@app.get("/account/history/{account_holder}")
def transaction_history(account_holder: str):
    bank_account = BankAccount(account_holder)
    history = bank_account.get_transaction_history()
    return {"account_holder": account_holder, "transaction_history": history}
